In [2]:
1+1

2

In [3]:
%pwd

'/Users/suyash/Desktop/End to End/research'

In [4]:
os.chdir('../')
%pwd

'/Users/suyash/Desktop/End to End'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_path: Path
    target_column: str
    all_params: dict
    mlflow_uri: str

In [6]:
from src.IPL_Predictor.constants import *
from src.IPL_Predictor.utils.common import read_yaml, create_directories
import joblib
import pandas as pd

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
        ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        create_directories([config.root_dir])

        evaluation_config = ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            test_data_path=Path(config.test_data_path),
            model_path=Path(config.model_path),
            metric_file_path=Path(config.metric_file_path),
            target_column=self.schema.TARGET_COLUMN,
            all_params=self.params.RandomForest,
            mlflow_uri="https://dagshub.com/suyashambule1234/End-to-End-IPL-prediction.mlflow"
        )

        return evaluation_config

In [9]:
import mlflow
import mlflow.sklearn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from urllib.parse import urlparse
import json

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluate_model(self):
        try:
            # Load test data and model
            test_data = pd.read_csv(self.config.test_data_path)
            model = joblib.load(self.config.model_path)

            # Prepare test data
            X_test = test_data.drop(self.config.target_column, axis=1)
            y_test = test_data[self.config.target_column]

            # Make predictions and calculate metrics
            predicted_qualities = model.predict(X_test)
            
            # Calculate metrics
            scores = {
                "accuracy": accuracy_score(y_test, predicted_qualities),
                "precision": precision_score(y_test, predicted_qualities, average='weighted'),
                "recall": recall_score(y_test, predicted_qualities, average='weighted'),
                "f1_score": f1_score(y_test, predicted_qualities, average='weighted')
            }

            # Save metrics locally first
            os.makedirs(os.path.dirname(self.config.metric_file_path), exist_ok=True)
            with open(self.config.metric_file_path, "w") as f:
                json.dump(scores, f, indent=4)
            
            # Try MLflow logging if configured
            try:
                mlflow.set_registry_uri(self.config.mlflow_uri)
                with mlflow.start_run():
                    # Log parameters
                    mlflow.log_params(self.config.all_params)
                    
                    # Log metrics
                    for metric_name, metric_value in scores.items():
                        mlflow.log_metric(metric_name, metric_value)
                    
                    # Log model
                    mlflow.sklearn.log_model(
                        model, 
                        "model", 
                        registered_model_name="RandomForestModel"
                    )
            except Exception as mlflow_error:
                print(f"Warning: MLflow logging failed: {str(mlflow_error)}")
                print("Continuing with local metric storage only")

            return scores

        except Exception as e:
            logger.error(f"Error in model evaluation: {str(e)}")
            raise e

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    
    scores = model_evaluation.evaluate_model()
    
    # Print results
    print("\nModel Evaluation Scores:")
    for metric, value in scores.items():
        print(f"{metric}: {value:.4f}")
        
except Exception as e:
    print(f"Error during model evaluation: {str(e)}")
    raise e

[2025-09-24 13:11:33,463: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-09-24 13:11:33,465: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-24 13:11:33,467: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-24 13:11:33,468: INFO: common: created directory at : artifacts]
[2025-09-24 13:11:33,468: INFO: common: created directory at : artifacts/model_evaluation]


2025/09/24 13:11:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run monumental-foal-732 at: https://dagshub.com/suyashambule1234/End-to-End-IPL-prediction.mlflow/#/experiments/0/runs/8705b78a6eb549cf97b2cd19eb57d55a
🧪 View experiment at: https://dagshub.com/suyashambule1234/End-to-End-IPL-prediction.mlflow/#/experiments/0
Continuing with local metric storage only

Model Evaluation Scores:
accuracy: 0.7201
precision: 0.7202
recall: 0.7201
f1_score: 0.7198
